In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset


In [3]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_tensors='pt',
            return_attention_mask=True,
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(self.labels[idx])
        }

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # Adjust num_labels based on your sentiment classes

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
dataset = load_dataset("financial_phrasebank",'sentences_50agree')

In [7]:
content = dataset['train']['sentence']
labels = dataset['train']['label']

In [8]:
train_data = SentimentDataset(content, labels, tokenizer)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

In [9]:
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
batch_size = 16

/Users/macsrini/Desktop/M.Tech/Data_Science_in_Practice/Project/project/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
print_interval = 1
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_idx,batch in enumerate(train_loader):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['label']
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        if batch_idx % print_interval == 0:
            average_loss = total_loss / (batch_idx + 1)
            print(f'Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_loader)}, Loss: {average_loss:.4f}')

    average_loss_epoch = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss_epoch:.4f}')


Epoch 1/3, Batch 1/303, Loss: 0.8315
Epoch 1/3, Batch 2/303, Loss: 0.9681
Epoch 1/3, Batch 3/303, Loss: 0.9644
Epoch 1/3, Batch 4/303, Loss: 0.9383
Epoch 1/3, Batch 5/303, Loss: 0.9501
Epoch 1/3, Batch 6/303, Loss: 0.9341
Epoch 1/3, Batch 7/303, Loss: 0.9142
Epoch 1/3, Batch 8/303, Loss: 0.8991
Epoch 1/3, Batch 9/303, Loss: 0.8770
Epoch 1/3, Batch 10/303, Loss: 0.8682
Epoch 1/3, Batch 11/303, Loss: 0.8640
Epoch 1/3, Batch 12/303, Loss: 0.8701
Epoch 1/3, Batch 13/303, Loss: 0.8662
Epoch 1/3, Batch 14/303, Loss: 0.8452
Epoch 1/3, Batch 15/303, Loss: 0.8468
Epoch 1/3, Batch 16/303, Loss: 0.8367
Epoch 1/3, Batch 17/303, Loss: 0.8442
Epoch 1/3, Batch 18/303, Loss: 0.8512
Epoch 1/3, Batch 19/303, Loss: 0.8419
Epoch 1/3, Batch 20/303, Loss: 0.8497
Epoch 1/3, Batch 21/303, Loss: 0.8369
Epoch 1/3, Batch 22/303, Loss: 0.8429
Epoch 1/3, Batch 23/303, Loss: 0.8417
Epoch 1/3, Batch 24/303, Loss: 0.8395
Epoch 1/3, Batch 25/303, Loss: 0.8331
Epoch 1/3, Batch 26/303, Loss: 0.8267
Epoch 1/3, Batch 27/3

In [14]:
filename="fine_tuned_sentiment_model.pt"
torch.save(model,filename)